<a href="https://colab.research.google.com/github/takeshitayy/openai-whisper/blob/main/openai_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 【OpenAI】Whisper で音声データを文章化し、発話の開始時間と終了時間を含めたCSVデータにする
- OpenAIが公開している [Wisper](https://github.com/openai/whisper/) を利用して、音声データを文章化する
- 文章化するとある程度で区切られて出力され、その区切られた文章の開始時間と終了時間を含めCSVデータにする

## 実行方法

1. ランタイムの設定
    1. メニューバーの [ランタイム] - [ランタイムのタイプの変更] をクリック
        ![ランタイムのタイプの変更](https://docs.google.com/drawings/d/e/2PACX-1vTo2XkKxxwux9ATDbSPQI0PbJHMYzTJPKwUcUtIj8UwlT_h4sFm9vJtdjusib3p0nNX-j7lYX0EyWu1/pub?w=601&h=545)
    1. ハードウェア アクセラレータ で `GUP` を設定する
        ![ランタイムのタイプの変更](https://docs.google.com/drawings/d/e/2PACX-1vQL7egTWuLu9NOLb5-Z1uZCwIa9dMKraLhn6T6B3KO0_GdYhawWdOyKQaE0SlBDZ0gDr1xAUp0aCPAQ/pub?w=644&h=340)
1. 音声データの準備
    1. サイドバーの 📁アイコン をクリックしてファイルの一覧を表示し、テキストを抽出したい 音声データ を drag & drop で追加
        ![音声データの追加](https://docs.google.com/drawings/d/e/2PACX-1vQU_evi3lR57aAgoRfJhAr5FYpGBf10uA85v8Dx25rE-HMAauXwX1ON0o9KcBVyXvUjqR4H8_rMadCM/pub?w=525&h=307)
    1. 追加した 音声データ を右クリックして `パスをコピー` でファイルの絶対パスをコピーする
    1. 変数 `audioPath` に音声データの絶対パスをセット
1. そのほか設定
    1. 変数 `outputCsvName` に出力するcsvのファイル名をセット
    1. 変数 `model`で 使用する whisper のモデルを選択する
1. `Ctrl + F9` ですべてのセルを実行すると

### 変数のセットとパッケージの導入

In [1]:
#@title 変数のセット
#@markdown 音声データの絶対パス:
audioPath = "/content/audio.mp3" #@param {type:"string"}

#@markdown 出力するCSVのファイル名（/content/ 以下に出力される）:
outputCsvName = "output.csv" #@param {type:"string"}

#@markdown whisper のモデル:
model = "large" #@param ["tiny", "base", "small", "medium", "large"]

import os
outputCsvPaht = os.path.join('/content/', outputCsvName)

#### パッケージの導入

In [2]:
! pip install git+https://github.com/openai/whisper.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-6785qgc1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-6785qgc1
  Resolved https://github.com/openai/whisper.git to commit b5851c6c40e753606765ac45b85b298e3ae9e00d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.4 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796955 sha256=b838d09b7a4d539c3e37718719523f7cf16af6a95ec4a588a88a8962f8a4536c
  Stored in directory: /tmp/pip-ephem-wheel-cache-9r8a86i4/wheels/fe/03/29/e7919208d11b4ab32972cb448bb84a9a675d92cd52c9a48341
Successfully built openai-whisper


## 音声データを文章化
Whisper で音声データの文章化を行うと以下の dict が返ってくる  

**language**: 検出された言語  
**text**: 文章全体  
**segments**: 分割された文章の詳細情報（その文章の開始時間や終了時間の情報も含む）


In [3]:
import whisper
model = whisper.load_model(model)
result = model.transcribe(audioPath)
print('言語: ' + result["language"])
print(result["text"])

100%|██████████████████████████████████████| 2.87G/2.87G [00:16<00:00, 190MiB/s]


言語: ja
宮沢賢治作 雨にも負けず雨にも負けず 風にも負けず雪にも 夏の暑さにも負けぬ丈夫な体を持ち 欲はなく決して怒らず いつも静かに笑っている一日に玄米四合と味噌と少しの 野菜を食べあらゆることを自分を感情に入れ ずによく見聞きしわかりそして忘れず野原の松の林の陰の小さなかや ぶきの小屋にいて東に病気の子供あれば 行って看病してやり西に疲れた母あれば 行ってその稲の束を追い南に死にそうな人あれば 行って怖がらなくてもいいといい北に喧嘩や訴訟があれば つまらないからやめろといい緑の時は涙を流し 寒さの夏はおろおろ歩きみんなにデクノボーと呼ばれ 褒められもせず苦にもされずそういうものに私はなりたい南無変行菩薩 南常行菩薩南多方如来 南明方蓮華経南釈迦無二仏 南常行菩薩南安隆行菩薩


## 発話の開始時間と終了時間を含めたCSVデータにする
segments のデータを csvデータにする


In [4]:
import pandas as pd
audioDf = pd.DataFrame.from_dict(result["segments"])

# csv ファイルに出力
audioDf.to_csv(outputCsvPaht)

# 内容の表示
audioDf.head()

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob
0,0,0,0.00,11.00,宮沢賢治作 雨にも負けず,"[50364, 39194, 3308, 95, 13352, 95, 30118, 119...",0.0,-0.21271,1.405797,0.211058
1,1,0,11.00,14.08,雨にも負けず 風にも負けず,"[50914, 35339, 4108, 4801, 11561, 254, 7625, 1...",0.0,-0.21271,1.405797,0.211058
2,2,0,14.08,19.52,雪にも 夏の暑さにも負けぬ,"[51068, 47939, 4108, 4801, 220, 42708, 2972, 2...",0.0,-0.21271,1.405797,0.211058
3,3,0,19.52,24.38,丈夫な体を持ち 欲はなく,"[51340, 39642, 3203, 29485, 5998, 17694, 6574,...",0.0,-0.21271,1.405797,0.211058
4,4,0,24.38,28.20,決して怒らず いつも静かに笑っている,"[51583, 33540, 8822, 3757, 240, 5154, 18216, 3...",0.0,-0.21271,1.405797,0.211058
